<a href="https://colab.research.google.com/github/machine-perception-robotics-group/GoogleColabNotebooks/blob/master/MLDL_lecture_notebooks/10-C_objecct_classification_with_cifar10_whitening_augmentation_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10-C. CIFAR10を用いた物体認識（ResNet）


---
## 目的
CIFAR10 Datasetを用いて10クラスの物体認識を行う．プログラムの構成は，MNISTによる文字認識のプログラムと同様になっているため，基礎的な説明はそちらを参照して頂きたい．このページでは，MNISTによる文字認識のプログラムとの差分について書いていく．

GPUを用いたネットワークの計算を行う．
また，Data Augmentationを用いた学習の効果について確認する．
ここでは畳み込みニューラルネットワークのモデルとして，ResNetを用いて実験を行う．

## 準備

### Google Colaboratoryの設定確認・変更
本チュートリアルではPyTorchを利用してニューラルネットワークの実装を確認，学習および評価を行います．
**GPUを用いて処理を行うために，上部のメニューバーの「ランタイム」→「ランタイムのタイプを変更」からハードウェアアクセラレータをGPUにしてください．**


## 使用するデータセット

### データセット
今回の物体認識では，CIFAR10データセットを用いる．CIFAR10データセットは，飛行機や犬などの10クラスの物体が表示されている画像から構成されたデータセットである．

![CIFAR10_sample.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/176458/b6b43478-c85f-9211-7bc6-227d9b387af5.png)

## モジュールのインポート
はじめに必要なモジュールをインポートする．

### GPUの確認
GPUを使用した計算が可能かどうかを確認します．

`GPU availability: True`と表示されれば，GPUを使用した計算をChainerで行うことが可能です．
Falseとなっている場合は，上記の「Google Colaboratoryの設定確認・変更」に記載している手順にしたがって，設定を変更した後に，モジュールのインポートから始めてください．


In [ ]:
# モジュールのインポート
from time import time
import numpy as np
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms

# GPUの確認
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

## データセットの読み込みと確認
学習データ（CIFAR10データセット）を読み込みます．

読み込んだ学習データのサイズを確認します．
学習データは5万枚，1つのデータサイズは3x32x32の画像のような形式となっています．
これは32x32ピクセルのカラー画像という意味になります．

In [ ]:
train_data = torchvision.datasets.CIFAR10(root="./", train=True, transform=transforms.ToTensor(), download=True)
test_data = torchvision.datasets.CIFAR10(root="./", train=False, transform=transforms.ToTensor(), download=True)

print(train_data)
print(test_data)

## ネットワークモデルの定義
Residual Network (ResNet) を定義します．




In [ ]:
class BottleNeck(nn.Module):
    def __init__(self, n_in, n_mid, n_out, stride=1, use_conv=False):
        super().__init__()
        
        self.use_conv = use_conv
        
        self.convs = nn.Sequential(*[nn.Conv2d(n_in, n_mid, kernel_size=1, stride=stride, padding=0),
                                     nn.BatchNorm2d(n_mid),
                                     nn.Conv2d(n_mid, n_mid, kernel_size=3, stride=1, padding=1),
                                     nn.BatchNorm2d(n_mid),
                                     nn.Conv2d(n_mid, n_out, kernel_size=1, stride=1, padding=0),
                                     nn.BatchNorm2d(n_out)
                                    ])
        if self.use_conv:
            self.conv4 = nn.Conv2d(n_in, n_out, kernel_size=1, stride=stride, padding=0)
            self.bn4 = nn.BatchNorm2d(n_out)
        self.act = nn.ReLU()
        
    def forward(self, x):
        h = self.convs(x)
        if self.use_conv:
            h = h + self.bn4(self.conv4(x))
        else:
            h = h + x
        return h
    

class ResidualBlock(nn.Module):
    def __init__(self, n_in, n_mid, n_out, n_bottlenecks, stride=2):
        super().__init__()
        layers = []
        
        layers.append(BottleNeck(n_in, n_mid, n_out, stride, True))
        for _ in range(n_bottlenecks - 1):
            layers.append(BottleNeck(n_out, n_mid, n_out))
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        h = self.layers(x)
        return h
    

class ResNet(nn.Module):
    def __init__(self, n_class=10, n_blocks=[3, 4, 6, 3]):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(64)
        self.act = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.res1 = ResidualBlock(64, 64, 256, n_blocks[0], 1)
        self.res2 = ResidualBlock(256, 128, 512, n_blocks[1], 2)
        self.res3 = ResidualBlock(512, 256, 1024, n_blocks[2], 2)
        self.res4 = ResidualBlock(1024, 512, 2048, n_blocks[3], 2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(2048, n_class)
        
    def forward(self, x):
        h = self.act(self.bn1(self.conv1(x)))
        h = self.res1(h)
        h = self.res2(h)
        h = self.res3(h)
        h = self.res4(h)
        h = self.avgpool(h)
        h = torch.flatten(h, 1)
        h = self.fc(h)
        return h
        
        
class ResNet50(ResNet):
    def __init__(self, n_class=10):
        super(ResNet50, self).__init__(n_class, n_blocks=[3, 4, 6, 3])

## ネットワークの作成
上のプログラムで定義したネットワークを作成します．

CNNクラスを呼び出して，ネットワークモデルを定義します． また，GPUを使う場合（use_cuda == True）には，ネットワークモデルをGPUメモリ上に配置します． これにより，GPUを用いた演算が可能となります．

学習を行う際の最適化方法としてモーメンタムSGD(モーメンタム付き確率的勾配降下法）を利用します． また，学習率を0.01，モーメンタムを0.9として引数に与えます．


In [ ]:
model = ResNet50(n_class=10)
if use_cuda:
    model.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

## 学習
１回の誤差を算出するデータ数（ミニバッチサイズ）を64，学習エポック数を100とします．
CIFAR10の学習データサイズを取得し，１エポック内における更新回数を求めます．
学習データは毎エポックでランダムに利用するため，numpyのpermutationという関数を利用します．
各更新において，学習用データと教師データをそれぞれxとtとし，to_gpu関数でGPUに転送します．
学習モデルにxを与えて各クラスの確率yを取得します．各クラスの確率yと教師ラベルtとの誤差をsoftmax coross entropy誤差関数で算出します．
また，認識精度も算出します．そして，誤差をbackward関数で逆伝播し，ネットワークの更新を行います．

In [ ]:
# ミニバッチサイズ・エポック数の設定
batch_size = 64
epoch_num = 10
n_iter = len(train_data) / batch_size

# データローダーの設定
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

# 誤差関数の設定
criterion = nn.CrossEntropyLoss()
if use_cuda:
    criterion.cuda()

# ネットワークを学習モードへ変更
model.train()

start = time()
for epoch in range(1, epoch_num+1):
    sum_loss = 0.0
    count = 0
    
    for image, label in train_loader:
        if use_cuda:
            image = image.cuda()
            label = label.cuda()

        y = model(image)        
        loss = criterion(y, label)
        
        model.zero_grad()
        loss.backward()
        optimizer.step()
        
        sum_loss += loss.item()
        
        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

    print("epoch: {}, mean loss: {}, mean accuracy: {}, elapsed_time :{}".format(epoch,
                                                                                 sum_loss / n_iter,
                                                                                 count.item() / len(train_loader),
                                                                                 time() - start))

## テスト
学習したネットワークモデルを用いて評価を行います．

In [ ]:
# データローダーの準備
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=False)

# ネットワークを評価モードへ変更
model.eval()

# 評価の実行
count = 0
with torch.no_grad():
    for image, label in test_loader:

        if use_cuda:
            image = image.cuda()
            label = label.cuda()
            
        y = model(image)

        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

print("test accuracy: {}".format(count.item() / 10000.))